<a href="https://colab.research.google.com/github/cfarhutchins/technical_direction_service/blob/master/BOW_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils



Using TensorFlow backend.


In [2]:
pip install GetOldTweets3

In [0]:
import GetOldTweets3 as got

In [0]:
tweetCriteria = got.manager.TweetCriteria().setQuerySearch('reparations')\
                                           .setSince('2019-07-04')\
                                           .setUntil('2019-07-10')\
                                           .setMaxTweets(30000)
tweet = got.manager.TweetManager.getTweets(tweetCriteria)

repar_tweets = []

for i in tweet:
  repar_tweets.append(i.text)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
pip install vaderSentiment

     |████████████████████████████████| 133kB 562kB/s 


In [0]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [0]:
analyzer = SentimentIntensityAnalyzer()

In [0]:
df_repar = pd.DataFrame(repar_tweets)
df_repar.rename(columns={0:'tweet'}, inplace= True)
df_repar['feels'] = df_repar['tweet'].apply(lambda i: analyzer.polarity_scores(i))
compound = []

for dct in df_repar['feels']:
  compound.append(dct['compound'])

negative = []
  
for dct in df_repar['feels']:
  negative.append(dct['neg'])
  
neutral = [] 
  
for dct in df_repar['feels']:
  neutral.append(dct['neu'])
  
positive = []

for dct in df_repar['feels']:
  positive.append(dct['pos'])
  
df_repar['compound'] = compound
df_repar['negative'] = negative
df_repar['neutral'] = neutral
df_repar['positive'] = positive

df_repar.drop(columns = 'feels', inplace = True)

In [83]:
df_repar.head()

,tweet,compound,negative,neutral,positive
0,He needs to shut up. He wants google to provid...,0.0000,0.000,1.000,0.000
1,@BarackObama was a good lesson for Black Ameri...,-0.5095,0.189,0.735,0.076
2,Where are my reparations?? https://www.globalr...,0.0000,0.000,1.000,0.000
3,But the 1 point that made the most sense which...,-0.8685,0.223,0.650,0.127
4,Let's talk reparations: https://youtu.be/qSF95...,0.0000,0.000,1.000,0.000


In [0]:
cv = CountVectorizer(ngram_range=(2,2),binary=True, stop_words='english')


In [0]:
x = df_repar['tweet']
y = df_repar['compound']



In [0]:
from sklearn.model_selection import train_test_split, cross_val_score

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= .33)

X_train = cv.fit_transform(x_train)

X_test = cv.transform(x_test)

In [0]:
from sklearn.linear_model import Ridge

rdg = Ridge()

In [89]:
rdg.fit(X_train, y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [90]:
rdg.predict(X_test)

array([ 0.03209193,  0.07796892, -0.14909093, ...,  0.03531878,
       -0.07548991, -0.09829616])

In [0]:
rdg.score(X_test, y_test)
pingwords = cv.get_feature_names()

In [0]:
df_pingwords = pd.DataFrame(rdg.coef_, index = pingwords).sort_values(by=0)


In [93]:
df_pingwords[0:10]

,0
descendants slavery,-0.520394
slavery reparations,-0.481669
reparations slavery,-0.465316
reparations stupid,-0.462062
hell reparations,-0.411688
reparations bullshit,-0.398558
civil war,-0.378630
pain suffering,-0.377374
reparations stealing,-0.360721
crazy shit,-0.352386


In [0]:
pingwords_lst = []

for i in pingwords:
  pingwords_lst.append(i)

In [95]:
print(df_repar['tweet'].apply(lambda x: len(x.split(' '))).sum())

677997


In [0]:
df_pingwords = df_pingwords.index
  

In [0]:
pingwords_lst = df_pingwords.to_list()

In [0]:
df_pingwords = pd.DataFrame(pingwords_lst)

In [99]:
df_pingwords

,0
0,descendants slavery
1,slavery reparations
2,reparations slavery
3,reparations stupid
4,hell reparations
5,reparations bullshit
6,civil war
7,pain suffering
8,reparations stealing
9,crazy shit


In [0]:
df_pingwords.rename(columns={0:'ping pair'}, inplace= True)

In [101]:
df_pingwords

,ping pair
0,descendants slavery
1,slavery reparations
2,reparations slavery
3,reparations stupid
4,hell reparations
5,reparations bullshit
6,civil war
7,pain suffering
8,reparations stealing
9,crazy shit


In [0]:
from sklearn.utils.validation import check_is_fitted
from sklearn.utils.validation import column_or_1d

In [0]:
class TolerantLabelEncoder(LabelEncoder):
    def __init__(self, ignore_unknown=False,
                       unknown_original_value='unknown', 
                       unknown_encoded_value=-1):
        self.ignore_unknown = ignore_unknown
        self.unknown_original_value = unknown_original_value
        self.unknown_encoded_value = unknown_encoded_value

    def transform(self, y):
        check_is_fitted(self, 'classes_')
        y = column_or_1d(y, warn=True)

        indices = np.isin(y, self.classes_)
        if not self.ignore_unknown and not np.all(indices):
            raise ValueError("y contains new labels: %s" 
                                         % str(np.setdiff1d(y, self.classes_)))

        y_transformed = np.searchsorted(self.classes_, y)
        y_transformed[~indices]=self.unknown_encoded_value
        return y_transformed

    def inverse_transform(self, y):
        check_is_fitted(self, 'classes_')

        labels = np.arange(len(self.classes_))
        indices = np.isin(y, labels)
        if not self.ignore_unknown and not np.all(indices):
            raise ValueError("y contains new labels: %s" 
                                         % str(np.setdiff1d(y, self.classes_)))

        y_transformed = np.asarray(self.classes_[y], dtype=object)
        y_transformed[~indices]=self.unknown_original_value
        return y_transformed

In [134]:
train_size = int(len(df_repar) * .7)
train_posts = df_repar['tweet'][:train_size]
train_tags = df_repar['negative'][:train_size]

test_posts = df_repar['tweet'][train_size:]
test_tags = df_repar['negative'][train_size:]

max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_posts) # only fit on train

x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)


encoder = TolerantLabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.fit_transform(test_tags)

num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

batch_size = 32
epochs = 3

# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
              
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 16022 samples, validate on 1781 samples
Epoch 1/3
16022/16022 [==============================] - 8s 487us/step - loss: 3.9045 - acc: 0.4395 - val_loss: 3.8086 - val_acc: 0.4116
Epoch 2/3
16022/16022 [==============================] - 8s 482us/step - loss: 3.3328 - acc: 0.4505 - val_loss: 3.7580 - val_acc: 0.4161
Epoch 3/3
16022/16022 [==============================] - 8s 505us/step - loss: 2.9862 - acc: 0.4607 - val_loss: 3.7817 - val_acc: 0.4166


In [0]:
#model.predict(x_test)

In [0]:
#model.evaluate(x_test, y_test, batch_size= batch_size, verbose = 1)
#print('Test accuracy:', score[0])

In [0]:
#score = model.evaluate(x_test, y_test,
#                       batch_size=batch_size, verbose=1)
#print('Test accuracy:', score[0])